In [6]:
import networkx as nx
from neo4j import GraphDatabase
import matplotlib.pyplot as plt

连接Neo4j

In [7]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

neo4j_conn = Neo4jConnection(uri="bolt://localhost:7687/", user="neo4j", pwd="zxysxtc233")

使用NetworkX建立图

In [8]:
query = """
MATCH (n)-[r]->(m)
RETURN n.name AS nName, ID(n) AS nId, m.name AS mName, ID(m) AS mId, r
"""
results = neo4j_conn.query(query)

results

Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


[<Record nName='(multiple sellers)' nId=0 mName='Brazil' mId=29 r=<Relationship element_id='8267' nodes=(<Node element_id='0' labels=frozenset() properties={}>, <Node element_id='29' labels=frozenset() properties={}>) type='supplied' properties={'order_year': '2019', 'num_delivered': 'Pending', 'weapon_description': 'BVRAAM', 'designation': 'Meteor', 'delivery_year': 'Pending'}>>,
 <Record nName='Albania' nId=4 mName='Burkina Faso' mId=32 r=<Relationship element_id='0' nodes=(<Node element_id='4' labels=frozenset() properties={}>, <Node element_id='32' labels=frozenset() properties={}>) type='supplied' properties={'order_year': '2011', 'num_delivered': '12', 'weapon_description': 'Mortar', 'designation': 'M-43 120mm', 'delivery_year': '2011'}>>,
 <Record nName='Algeria' nId=5 mName='Angola' mId=7 r=<Relationship element_id='1' nodes=(<Node element_id='5' labels=frozenset() properties={}>, <Node element_id='7' labels=frozenset() properties={}>) type='supplied' properties={'order_year': 

In [9]:
import csv
import os 

csv_file_path = "graph_data/total.csv"
csv_dir = os.path.dirname(csv_file_path)

if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Source", "Target"])
    for record in results:
        writer.writerow([record["nName"], record["mName"]])

In [10]:
G = nx.MultiDiGraph()

# 添加节点和边
for record in results:
    n_id = record["nId"]
    m_id = record["mId"]
    n_name = record["nName"]
    m_name = record["mName"]
    r_attributes = dict(record["r"].items())  # 假设r是一个包含属性的Neo4j Record对象

    # 添加节点，使用节点的name属性作为标签
    G.add_node(n_id, label=n_name)
    G.add_node(m_id, label=m_name)

    # 添加边，标签设为"supplied"
    G.add_edge(n_id, m_id, label="supplied", **r_attributes)

# 关闭Neo4j连接
neo4j_conn.close()

In [11]:
G.number_of_nodes(), G.number_of_edges()

(265, 26631)

度排序

In [12]:
# 计算所有节点的入度和出度
in_degrees = dict(G.in_degree())
out_degrees = dict(G.out_degree())

# 按入度降序排序节点，并获取前10个
top_10_in_degrees = sorted(in_degrees, key=in_degrees.get, reverse=True)[:10]
# 按出度降序排序节点，并获取前10个
top_10_out_degrees = sorted(out_degrees, key=out_degrees.get, reverse=True)[:10]

# 获取这些节点的name，id和度数
# 注意这里将 'name' 改为 'label' 以匹配你实际设置的属性键
top_10_in_degrees_info = [(node, G.nodes[node]['label'], in_degrees[node]) for node in top_10_in_degrees]
top_10_out_degrees_info = [(node, G.nodes[node]['label'], out_degrees[node]) for node in top_10_out_degrees]

In [13]:
# 输出入度最高的10个节点的信息
for info in top_10_in_degrees_info:
    print(f"Node ID: {info[0]}, Name: {info[1]}, In-Degree: {info[2]}")

Node ID: 98, Name: India, In-Degree: 718
Node ID: 62, Name: Egypt, In-Degree: 615
Node ID: 228, Name: Thailand, In-Degree: 562
Node ID: 234, Name: Turkey, In-Degree: 559
Node ID: 84, Name: Greece, In-Degree: 541
Node ID: 99, Name: Indonesia, In-Degree: 502
Node ID: 200, Name: Saudi Arabia, In-Degree: 501
Node ID: 175, Name: Pakistan, In-Degree: 486
Node ID: 211, Name: South Korea, In-Degree: 474
Node ID: 102, Name: Iraq, In-Degree: 456


In [14]:
# 输出出度最高的10个节点的信息
for info in top_10_out_degrees_info:
    print(f"Node ID: {info[0]}, Name: {info[1]}, Out-Degree: {info[2]}")

Node ID: 245, Name: United States, Out-Degree: 8612
Node ID: 216, Name: Soviet Union, Out-Degree: 3358
Node ID: 77, Name: France, Out-Degree: 2407
Node ID: 243, Name: United Kingdom, Out-Degree: 1904
Node ID: 82, Name: Germany, Out-Degree: 1201
Node ID: 192, Name: Russia, Out-Degree: 1129
Node ID: 105, Name: Italy, Out-Degree: 924
Node ID: 41, Name: China, Out-Degree: 903
Node ID: 104, Name: Israel, Out-Degree: 642
Node ID: 158, Name: Netherlands, Out-Degree: 554


PageRank

In [15]:
pagerank = nx.pagerank(G)

pagerank_with_labels = {G.nodes[node_id]['label']: rank for node_id, rank in pagerank.items()}

pagerank_with_labels


{'(multiple sellers)': 0.0017038623514746575,
 'Brazil': 0.006563376252882768,
 'Albania': 0.002005952775740612,
 'Burkina Faso': 0.011517865385026332,
 'Algeria': 0.004687070434797691,
 'Angola': 0.012460969893983766,
 'Nigeria': 0.00919227745679946,
 'Western Sahara': 0.0032810856308873293,
 'Congo': 0.005540990041216654,
 'Cote dIvoire': 0.006564229877324563,
 'Guinea-Bissau': 0.004277630058426135,
 'United States': 0.013795768205557666,
 'Argentina': 0.007836115698814372,
 'Bolivia': 0.01702980156336316,
 'Colombia': 0.005651722166824771,
 'Denmark': 0.004587380590387571,
 'Ecuador': 0.008364494547996058,
 'Honduras': 0.005059589003778422,
 'Paraguay': 0.006744272096863903,
 'Spain': 0.005606090311907688,
 'Sri Lanka': 0.0064250147366245215,
 'Uruguay': 0.0145407649873836,
 'Armenia': 0.0020360887953068062,
 'Sudan': 0.010956686250908005,
 'Aruba': 0.0017038623514746575,
 'Israel': 0.005505853178727605,
 'Australia': 0.004671550884446459,
 'Lesotho': 0.0019432271108291868,
 'Yemen'

In [16]:
max_pagerank_node_label = G.nodes[max(pagerank, key=pagerank.get)]['label']

max_pagerank_node_label

top_3_pagerank = sorted(pagerank, key=pagerank.get, reverse=True)[:3]
top_3_pagerank_info = [(node, G.nodes[node]['label'], pagerank[node]) for node in top_3_pagerank]

max_pagerank_node_label, top_3_pagerank_info


('Bolivia',
 [(26, 'Bolivia', 0.01702980156336316),
  (251, 'Uruguay', 0.0145407649873836),
  (245, 'United States', 0.013795768205557666)])

between central

In [17]:
between_central= nx.betweenness_centrality(G)

between_central_labels = {G.nodes[node_id]['label']: rank for node_id, rank in between_central.items()}
between_central_labels

{'(multiple sellers)': 0.0,
 'Brazil': 0.010692004150668536,
 'Albania': 8.298629971633774e-06,
 'Burkina Faso': 0.0,
 'Algeria': 0.004360105496151428,
 'Angola': 0.014573538482667635,
 'Nigeria': 0.00029282430686831213,
 'Western Sahara': 0.0,
 'Congo': 0.0,
 'Cote dIvoire': 0.00047232928104993345,
 'Guinea-Bissau': 0.0,
 'United States': 0.11581534061399061,
 'Argentina': 0.0008378927516155533,
 'Bolivia': 0.0,
 'Colombia': 0.0037282822080842103,
 'Denmark': 0.0064215280844349566,
 'Ecuador': 0.00014737705931967943,
 'Honduras': 0.0,
 'Paraguay': 0.0,
 'Spain': 0.015645000322416763,
 'Sri Lanka': 0.0005341565000655911,
 'Uruguay': 0.00010290215635013421,
 'Armenia': 4.8008603141682986e-06,
 'Sudan': 0.008107708999023305,
 'Aruba': 0.0,
 'Israel': 0.01235905280929025,
 'Australia': 0.013044108563520222,
 'Lesotho': 0.0,
 'Yemen': 0.0,
 'Vanuatu': 0.0,
 'United Kingdom': 0.03612501682685533,
 'Tuvalu': 0.0,
 'Trinidad and Tobago': 0.0,
 'Tonga': 0.0,
 'Timor-Leste': 0.0,
 'Thailand': 7

In [18]:
max_between_central_label = G.nodes[max(between_central, key=between_central.get)]['label']

top_3_between_central = sorted(between_central, key=between_central.get, reverse=True)[:3]
top_3_between_central_info = [(node, G.nodes[node]['label'], between_central[node]) for node in top_3_between_central]

max_between_central_label, top_3_between_central_info

('United States',
 [(245, 'United States', 0.11581534061399061),
  (243, 'United Kingdom', 0.03612501682685533),
  (77, 'France', 0.03579203248328982)])

In [21]:
neo4j_conn = Neo4jConnection(uri="bolt://localhost:7687/", user="neo4j", pwd="zxysxtc233")

query = """
MATCH (n)-[r]->(m)
WHERE r.delivery_year >= '1950' AND r.delivery_year <= '1959'
RETURN n.name AS Source, m.name AS Target
"""
results = neo4j_conn.query(query)

csv_file_path = "graph_data/1950s.csv"

csv_dir = os.path.dirname(csv_file_path)

if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Source", "Target"])  # 写入表头
    
    for record in results:
        writer.writerow([record["Source"], record["Target"]])

neo4j_conn.close()

In [22]:
neo4j_conn = Neo4jConnection(uri="bolt://localhost:7687/", user="neo4j", pwd="zxysxtc233")

query = """
MATCH (n)-[r]->(m)
WHERE r.delivery_year >= '2010' AND r.delivery_year <= '2019'
RETURN n.name AS Source, m.name AS Target
"""
results = neo4j_conn.query(query)

csv_file_path = "graph_data/2010s.csv"

csv_dir = os.path.dirname(csv_file_path)

if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Source", "Target"])  # 写入表头
    
    for record in results:
        writer.writerow([record["Source"], record["Target"]])

neo4j_conn.close()

In [25]:
neo4j_conn = Neo4jConnection(uri="bolt://localhost:7687/", user="neo4j", pwd="zxysxtc233")

query = """
MATCH (n)-[r]->(m)
WHERE n.name = 'China' or m.name = 'China'
RETURN n.name AS Source, m.name AS Target
"""
results = neo4j_conn.query(query)

csv_file_path = "graph_data/China.csv"

csv_dir = os.path.dirname(csv_file_path)

if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Source", "Target"])  # 写入表头
    
    for record in results:
        writer.writerow([record["Source"], record["Target"]])

neo4j_conn.close()

In [24]:
neo4j_conn = Neo4jConnection(uri="bolt://localhost:7687/", user="neo4j", pwd="zxysxtc233")

query = """
MATCH (n)-[r]->(m)
WHERE m.name = 'China' 
RETURN n.name AS Source, m.name AS Target
"""
results = neo4j_conn.query(query)

csv_file_path = "graph_data/to_China.csv"

csv_dir = os.path.dirname(csv_file_path)

if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Source", "Target"])  # 写入表头
    
    for record in results:
        writer.writerow([record["Source"], record["Target"]])

neo4j_conn.close()